In [1]:
# Atari space invaders 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import gym
#from utils import plotlearning

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [3]:

#CNN network architecture
class DQNetwork(nn.Module):
    def __init__(self):
        super(DQNetwork,self).__init__()
        self.conv1 = nn.Conv2d(1,32,8,stride=4,padding=1)
        self.conv2 = nn.Conv2d(32,64,4,stride=2)
        self.conv2 = nn.Conv2d(64,128,3)
        self.fc1   = nn.Linear(128*19*8,512)
        self.fc2   = nn.Linear(512,6)
    
    def forward(self,observation):
        observation = torch.Tensor(observation)
        observation = observation.view(-1,1,185,95)
        observation = F.Relu(self.conv1(observation))
        observation = F.Relu(self.conv2(observation))
        observation = F.Relu(self.conv3(observation))
        observation = observation.flatten()
        observation = F.Relu(self.fc1(observation))
        actions     = self.fc2(observation)
        return actions
        
        
    

In [4]:
DQ=DQNetwork()
DQ.to(device)

DQNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=19456, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=6, bias=True)
)

In [5]:
#Optimiser and loss function
optimiser = optim.RMSprop(DQ.parameters(),lr=0.003)
loss      = nn.MSELoss()

In [6]:
#agent

class Agent(object):
    def __init__(self,gamma,epsilon,Max_Memory_Size,limit=0.05,replace=10000,actionspace=[0,1,2,3,4,5]):
        self.gamma = gamma
        self.epsilon = epsilon
        self.limit = limit
        self.actionspace = actionspace
        self.mem = Max_Memory_Size
        self.steps = 0
        self.learn_step_counter = 0
        self.memory = []
        self.mem_counter = 0
        self.replcae_target = replace
        self.Q_curr = DQ
        self.Q_succ = DQ
    def store_mmry_transition(self,state,action,reward,succ_state):
        if self.mem_counter<self.mem:
            self.memory.append([state,action,reward,succ_state])
        else:
            self.memory[self.mem_counter % self.mem] = [state,action,reward,succ_state]
            self.mem_counter+=1
    def chooseAction(self,observation):
        rand_action = np.random.random()
        actions = self.Q_curr.forward(observation)
        if rand < 1-self.epsilon:
            action = torch.argmax(actions[1]).item()
        else:
            action = np.random.choice(self.actionspace)
        self.steps +=1
        
        return action
    def Agent_learn(self,batch_size):
        self.Q_curr.optimser.zero_grad()
        if self.replace_target is not None and self.learn_step_counter % self.replace_target == 0 :
            self.Q_succ.load_state_dict(self.Q_curr.state_dict())
            
        if self.mem_counter + batch_size < self.mem:
            memstart = int(np.random.choice(range(self.mem_counter)))
        else:
            memstart = int(np.random.choice(range(self.mem_counter-batch_size-1)))
            
        minibatch = self.memory[memstart:memstart+batch_size]
        memory = np.array(minibatch)
        
        Q_pred = self.Q_curr.forward(list(memory[:,0][:])).to(device)
        Q_succ = self.Q_succ.forward(list(memory[:,3][:])).to(device)
        
        bestAction = torch.argmax(Q_succ,dim=1).to(device)
        rewards = torch.tensor(list(memory[:,2])).to(device)
        Qtarget = Q_pred
        Qtarget[:,bestAction] = rewards + self.gamma*torch.max(Q_curr[1])
        
        if self.steps > 500:
            if self.epsilon - 1e-4 > self.limit:
                self.epsilon -= 1e-4
            else:
                self.epsilon = self.limit
        loss = self.Q_curr.loss(Qtarget,Q_pred).to(device)
        loss.backward()
        self.Q_curr.optimiser.step()
        self.learn_step_counter += 1
        
        
        
        

In [ ]:
env = gym.make('SpaceInvaders-v0')
player= Agent(gamma=0.95,epsilon=1.0,Max_Memory_Size=5000,replace=None)
while player.mem_counter < player.mem:
    observation = env.reset()
    finished = False
    while not finished:
        action = env.action_space.sample()
        observation_new,reward,done,info = env.step(action)
        if done and info['ale.lives'] ==0:
            reward =-100
        player.store_mmry_transition(np.mean(observation[15:200,30:125],axis=2),action,reward,np.mean(observation_new[15:200,30:125],axis=2))
        observation = observation_new
        
print('populated memory with random actions')
        
scores=[]
epsilon_history =[]
num_games = 50
batch_size = 32
        
for i in range(num_games):
    print('starting game' ,i+1, 'epsilon:%.4f' %model.epsilon)
    epsilon_history.append(model.epsilon)
    observation = env.reset()
    frames = [np.sum(observation[15:200,30:125],axis=2)]
    score = 0
    lastAction = 0

    while not done:
        if(len(frames)==4):
            action = player.chooseAction(frames)
            frames=[]
        else:
            action = lastAction
        observation_new,reward,done,info = env.step(action)
        score += reward
        frames.append(np.sum(observation[15:200,30:125],axis=2))
        if done and info['ale.lives']==0:
            reward =-100
        player.store_mmry_transition(np.mean(observation[15:200,30:125],axis=2),action,reward,np.mean(observation_new[15:200,30:125],axis=2))
        observation = observation_new
        player.learn(batch_size)
        lastAction = action
        env.render()
    scores.appned(score)
    print('score:',score)

